In [ ]:
# INSTALLATION
import pandas as pd
import numpy as np
import openpyxl
import requests
from io import BytesIO
from io import StringIO
from datetime import date, datetime, timezone, timedelta
!pip install datawrapper
from datawrapper import Datawrapper
DW_SECRET = os.getenv('dw_secret')
dw = Datawrapper(access_token=DW_SECRET)
# Basis-URL für die Excel-Dateien
base_url = "https://public.eex-group.com/eex/eua-auction-report/emission-spot-primary-market-auction-report-{year}-data.xlsx"

# Aktuelles Jahr abrufen
current_year = datetime.now().year

# Erstes Jahr festlegen
first_year = 2021

# Liste für die Datenframes
dfs = []

# Loop über die Jahre
for year in range(current_year, first_year - 1, -1):
    # URL für das aktuelle Jahr erstellen
    current_url = base_url.format(year=year)

    # Überprüfen, ob die URL existiert
    response = requests.head(current_url)
    if response.status_code == 200:
        # Excel-Datei herunterladen
        response = requests.get(current_url)
        excel_data = BytesIO(response.content)

        # Daten aus Excel-Datei einlesen
        current_df = pd.read_excel(excel_data, header=5, engine='openpyxl')

        # Datenframe zur Liste hinzufügen
        dfs.append(current_df)

# Datenframes zusammenführen
df = pd.concat(dfs, ignore_index=True)

# Filtern der Daten
df = df[df["Auction Name"] == "Auction 4. Period DE"]

# Auswahl relevanter Spalten auswählen
df = df[["Date", "Auction Price €/tCO2", "Minimum Bid €/tCO2", "Maximum Bid €/tCO2"]]

# Umbenennen der Spalten
df = df.rename(columns={
    "Date": "Datum",
    "Auction Price €/tCO2": "Auktionspreis €/tCO2",
    "Minimum Bid €/tCO2": "Minimalgebot €/tCO2",
    "Maximum Bid €/tCO2": "Maximalgebot t/CO2"
})

# Aktuelles Datum und Uhrzeit
now = datetime.now()

# Berechnung des Datums vor zwei Jahren
two_years_ago = now - timedelta(days=2*365)

# Festlegen des ersten Tages des Monats vor zwei Jahren
start_date = two_years_ago.replace(day=1)

# Umwandlung der Spalte "Datum" in ein Datetime-Objekt
df['Datum'] = pd.to_datetime(df['Datum'], format='%Y-%m-%d')

# Formatierung des Zeitstempels
timestamp = now.strftime('%y%m%d')

print(df)

# Update des Charts
CHART_ID = "7MO47"
dw.add_data(chart_id=CHART_ID, data=df)
dw.publish_chart(chart_id=CHART_ID, display=False)